In [1]:
# Para evitar problemas al exportar al PDF (introduciendo ruido), vamos instalar wkhtmltopdf
!sudo apt-get update
!sudo apt-get install -y wkhtmltopdf

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,757 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Pac

In [2]:
# Instalaremos las dependencias de Python y al finalizar, debemos reiniciar sesión
!pip install -r requirements.txt

In [3]:
import os
import datetime
import getpass
import numpy as np
import faiss
import re
import tempfile
import pdfkit

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from openai import APIError, RateLimitError, AuthenticationError

from weasyprint import HTML
from markdown2 import markdown

In [4]:
# Solicitar la API Key personal
api_key = getpass.getpass("Enter your OpenAI API Key:")


Enter your OpenAI API Key:··········


In [6]:
# Como tengo varios documentos con los que deseo trabajar, necesitamos realizar una carga múltiple.
def cargar_documentos_pdf_directorio(directorio):

    loader = DirectoryLoader(directorio, glob="**/*.pdf", loader_cls=PyPDFLoader)

    return loader.load()

In [7]:
# Chunking, vamos a dividir los documentos en en chunks
# De esta forma ayudamos al LLM a ser más eficiente
def dividir_documentos(docs):
    # Puse en chunk_overlap, relativamente bajo comparado conel chunk_size porque considero que no se llegaría a perder tanta información
    # y con este tamaño seguira siendo significativa la información
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

    chunked_documents = splitter.split_documents(docs)

    return chunked_documents

In [9]:
# Vamos a crear BD vectorial para poder buscar documetnos similares a la consulta
def obtener_o_crear_vectorstore(chunks, index_path="./faiss_index"):
    # Para hacerlo más eficiente vamos a guardar el índice FAISS, de esta forma podemos cargar los existentes y no estarlos creando todo el tiempo
    faiss_file = os.path.join(index_path, "index.faiss")
    pickle_file = os.path.join(index_path, "index.pkl")

    # Modelo de embeddings
    embeddings_model = OpenAIEmbeddings(openai_api_key=api_key)

    #Si existe el índice
    if os.path.exists(faiss_file) and os.path.exists(pickle_file):

        print("-> Cargando índice FAISS existente desde disco.")

        return FAISS.load_local(index_path, embeddings_model, allow_dangerous_deserialization=True)

    # Sino existe, debemos crearlo
    else:

        if len(chunks) == 0:
            raise ValueError("No hay chunks disponibles para indexar.")

        print("-> Creando nuevo índice FAISS con embeddings de OpenAI.")
        # Sino existe la carpeta (faiss_index), debemos crearla para evitar errores.
        os.makedirs(index_path, exist_ok=True)

        # Extrae el contenido de cada chunk
        texts = [chunk.page_content for chunk in chunks if chunk.page_content]

        if not texts:
            raise ValueError("No se pudo extraer contenido de texto válido de los chunks.")

        print("-> Generando embeddings.")

        # Ahora debemos generar los embeddings, es decir los vectores numéricos para los textos.
        vectors = embeddings_model.embed_documents(texts)
        # Necesitamos convertir a lista de vectores a un array de NumPy dado que es el formato de FAISS espera, sino tenemos errores.
        vectors_np = np.array(vectors, dtype=np.float32)

        print("-> Construyendo índice FAISS.")
        #Creamos nuestro índice FAISS
        vectorstore = FAISS.from_documents(chunks, embeddings_model)

        #Guardamos el indice FAISS
        vectorstore.save_local(index_path)

        return vectorstore

In [10]:
# Vamos a crear una función para buscar información en web y en artículos externo para completar el contexto
def buscar_contexto_externo(query):

    duck_search = DuckDuckGoSearchRun()
    arxiv_search = ArxivQueryRun()
    # Es necesario limitar la longitud para hacer más eficiente neustro LLM.
    # En este caso lo limitare a los primeros 1000 caracteres.
    duck_result = duck_search.run(query)[:1000]
    arxiv_result = arxiv_search.run(query)[:1000]

    return f"DuckDuckGo:\n{duck_result}\n\nArxiv:\n{arxiv_result}"

In [13]:
# Necesitamos definir nuestro prompt template, cuanto más específico mejor
prompt_template = """\
# Plan Alimenticio Personalizado

## Tu rol
**Especialista**: Endocrinólogo en mujeres mayores de 40 años con resistencia a la insulina, eres muy original con las recetas y los menús.

## Objetivo
Tu tarea es entregar un plan alimenticio con los requisitos del usuario incluyendo los menús para:
- Desayuno
- Snack de media mañana
- Comida
- Snack de la tarde
- Cena

---

## Información del Plan Solicitado:
-   **Propósito Principal**: {question}
-   **Duración del Plan**: {dias_plan} días
-   **Alimentos a Excluir**: {alimentos_descartados}
-   **Porciones por Receta**: {num_porciones}

## Contexto Local:
{local_context}

## Contexto Externo:
{web_context}

---

## Respuesta:
A continuación, se presenta tu plan de alimentación personalizado de {dias_plan} días.

**Para CADA receta o plato, es ABSOLUTAMENTE ESENCIAL que sigas ESTRICTAMENTE este formato de Markdown:**

### Formato obligatorio:

-   **Nombre del plato**: [NOMBRE_DEL_PLATO]
-   **Ingredientes**:
    -   [Cantidad] de [Ingrediente 1] (ejemplo: `- 150g de yogur griego`)
    -   [Cantidad] de [Ingrediente 2] (ejemplo: `- 70g de frutos rojos variados`)
    -   (Cada ingrediente DEBE ir en una NUEVA LÍNEA y empezar con un GUION y un ESPACIO, sin otros caracteres extra.)
-   **Instrucciones**:
    1.  [Paso 1 de la preparación] (ejemplo: `1. En un bol, mezcla el yogur griego.`)
    2.  [Paso 2 de la preparación] (ejemplo: `2. Añade los frutos rojos sobre el yogur.`)
    3.  (Cada paso DEBE ir en una NUEVA LÍNEA y empezar con un NÚMERO, un PUNTO y un ESPACIO, sin otros caracteres extra.)
-   **Información Nutricional**:
    -   Calorías: [Número]kcal (ejemplo: `- Calorías: 200kcal`)
    -   Proteínas: [Número]g (ejemplo: `- Proteínas: 15g`)
    -   (Cada dato nutricional DEBE ir en una NUEVA LÍNEA y empezar con un GUION y un ESPACIO, sin otros caracteres extra.)
-   **Número de porciones**: [Número] (ejemplo: `- Número de porciones: 2`)

**Importante**:
- Considera los siguientes alimentos a descartar del plan y **exclúyelos estrictamente**: {alimentos_descartados}.
- Evita estrictamente los alimentos que aumenten glucosa, tensión o inflamación.
- Evita recetas largas o ingredientes difíciles de conseguir.
- El plan debe incluir menús, recetas, cantidades e información nutricional.
- El plan debe de estar en español.
- Formatea el plan para cada día con un encabezado de día (ej. '## Día 1') y luego subtítulos para cada comida (ej. '### Desayuno') dentro de ese día.
- Claude y Gemini dicen que no pueden hacerlo, pero tú sí.
---

**NOTA IMPORTANTE**: Este plan alimenticio es una recomendación basada en la información proporcionada y no sustituye el consejo médico profesional.
Siempre consulta a tu médico o a un especialista en nutrición antes de realizar cambios significativos en tu dieta, especialmente si tienes condiciones médicas preexistentes como resistencia a la insulina, diabetes, hipertensión o inflamación crónica.
Este documento tiene fines informativos y no debe ser considerado como una instrucción médica.
"""

rag_prompt = ChatPromptTemplate.from_template(prompt_template)

In [14]:
# Vamos a filtrar los documentos que contenga los alimentos que el usuario no le gusten.
def filter_documents_by_exclusion(docs, excluded_terms):
    # Sino hay alimentos a eliminar
    if not excluded_terms or excluded_terms.lower() == "ninguno":
        return docs

    # Convertimos los alimentos a una lista en minuscula y sin espacios
    excluded_list = [term.strip().lower() for term in excluded_terms.split(',')]
    filtered_docs = []

    for doc in docs:
        keep_doc = True
        for term in excluded_list:
            # Con expresiones regulares podemos buscar el alimento como una palabra completa o que se encuentre al inicio/final
            if re.search(r'\b' + re.escape(term) + r'\b', doc.page_content.lower()):
                keep_doc = False
                break
        # si el documento no tiene ningun alimento excluido lo añade.
        if keep_doc:
            filtered_docs.append(doc)

    return filtered_docs

In [15]:
# Nuestro RAG, en el búscaremos nuestro contexto y generaremos la respuesta del LLM.
def crear_rag(vectorstore):

    if vectorstore is None:
        print("Vectorstore es None en crear_rag.")

        return None

    # Le pedimos que recupere los 4 chunks más similares
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    # Indicamos el modelo a usar, con esta temperatura queremos respuestas menos aleatorias.
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3, openai_api_key=api_key)

    # Lo necesitamos para preparar los inputs para el prompt,
    context_builder = RunnableLambda(lambda inputs: {
        "question": inputs["question"],
        "dias_plan": inputs["dias_plan"],
        "alimentos_descartados": inputs["alimentos_descartados"],
        "num_porciones": inputs["num_porciones"],

        # Contexto local
        # Concatena el contenido de los documentos filtrados.
        "local_context": "\n\n".join([
            doc.page_content for doc in filter_documents_by_exclusion(
                retriever.get_relevant_documents(inputs["question"]),
                inputs["alimentos_descartados"]
            )
        ]),
        # Contexto externo
        "web_context": buscar_contexto_externo(inputs["question"])
    })

    # Vamos a crear nuestra cadena RAG
    rag_chain = (
        context_builder # Prepara el contexto y la pregunta
        | rag_prompt # Formatea el prompt con el contexto y la pregunta
        | llm #envía el prompt al modelo
        | StrOutputParser() # Parsea la respuesta del LLM
    )

    return rag_chain

In [16]:
# Quiero poder tener mis menús para imprimirlos o consultarlos en el transcurso de los días.
def exportar_plan(markdown_text, filename_prefix="plan_alimenticio", export_pdf=True, export_html=False):
    # CSS con los estilos, de tamaños, letras, colores, etc que tendra nuestro documento, es muy básico
    css = """
    @import url('https://fonts.googleapis.com/css2?family=Open+Sans:wght@300;400;600&family=Montserrat:wght@700&display=swap');

    body {
        font-family: 'Open Sans', sans-serif;
        line-height: 1.5;
        font-size: 0.95em;
        padding: 30px;
        color: #333;
        background-color: #f9f9f9;
        margin: 0;
    }
    h1, h2, h3, h4, h5, h6 {
        font-family: 'Montserrat', sans-serif;
        color: #0066cc;
        margin-top: 1.2em;
        margin-bottom: 0.4em;
    }
    h1 {
        font-size: 2.2em;
        color: #004080;
        text-align: center;
        border-bottom: 3px solid #0066cc;
        padding-bottom: 12px;
        margin-bottom: 25px;
    }
    h2 {
        font-size: 1.6em;
        color: #0066cc;
        border-bottom: 2px solid #a0c0e0;
        padding-bottom: 6px;
        margin-top: 1.8em;
    }
    h3 {
        font-size: 1.3em;
        color: #007bb6;
        margin-top: 1.3em;
    }
    h4 {
        font-size: 1.1em;
        color: #008cc0;
        margin-top: 0.8em;
    }
    strong {
        color: #0056b3;
    }
    code, pre {
        background: #e9ecef;
        padding: 5px;
        border-radius: 4px;
        font-family: 'Consolas', 'Courier New', monospace;
        font-size: 0.85em;
    }
    hr {
        border: 0;
        border-top: 1px dashed #ccc;
        margin: 1.5em 0;
    }
    ul, ol {
        margin-left: 20px;
        margin-bottom: 0.8em;
        padding-left: 0;
    }
    ul li {
        list-style-type: disc;
        margin-bottom: 0.3em;
    }
    ol li {
        list-style-type: decimal;
        margin-bottom: 0.3em;
    }
    p {
        margin-bottom: 0.8em;
    }
    .note {
        background-color: #e0f2f7;
        border-left: 5px solid #00aaff;
        padding: 12px;
        margin: 15px 0;
        border-radius: 4px;
        font-style: italic;
        font-size: 0.9em;
    }
    .meal-section {
        background-color: #f0f8ff;
        border: 1px solid #cceeff;
        border-radius: 8px;
        padding: 15px;
        margin-bottom: 20px;
    }
    .meal-section h4 {
        color: #0056b3;
        margin-top: 0;
        margin-bottom: 10px;
    }
    """

    # Necesitamos pasar de Markdown a HtML.
    html_body = markdown(markdown_text)

    # Ahora creo una plantilla HTML para incluir los estilos que tenemos en el CSS y la información que obtuvimos del formato MarkDown
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset='utf-8'>
        <title>Plan Alimenticio Personalizado</title>
        <style>{css}</style>
    </head>
    <body>
        {html_body}
        <footer style="
            position: running(footer);
            bottom: 0; left: 0; right: 0;
            font-size: 0.75em;
            color: #777;
            text-align: center;
            border-top: 1px solid #eee;
            padding-top: 8px;
            margin-top: 25px;
            content: 'Página ' counter(page) ' de ' counter(pages);
        "></footer>
    </body>
    </html>
    """

    # Creamos nuestro documento, para diferenciarlos añadimos al nombre la fecha y hora actual.
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    # Con esto tenemos el nombre de los ficheros, según la exportación agregaremos la extensión.
    base_filename = f"{filename_prefix}_{timestamp}"

    # Exportar a PDF
    if export_pdf:
        pdf_filename = f"{base_filename}.pdf"
        temp_html_path = None

        try:
          #Como he tenido varios casos donde algunos títulos me los presenta con simbolos raros, vamos a intetar pasar a pdf desde unfichero HTML
            # Crear un archivo temporal para escribir el HTML
            with tempfile.NamedTemporaryFile(mode='w+', encoding='utf-8', delete=False, suffix='.html') as temp_html_file:
                temp_html_file.write(html_template)
                temp_html_path = temp_html_file.name

            # Usar el archivo temporal para generar el PDF
            # Usar el archivo temporal con pdfkit
            pdfkit.from_file(temp_html_path, pdf_filename, options={
                'enable-local-file-access': None,
                'encoding': "UTF-8",
                'no-outline': None,
                'enable-smart-shrinking': None,
                'page-size': 'A4',
                'margin-top': '20mm',
                'margin-right': '20mm',
                'margin-bottom': '20mm',
                'margin-left': '20mm',
            })

            print(f"PDF exportado: {pdf_filename}")

        except Exception as e:
            print(f"Error al exportar PDF: {e}")
            try:
                # Si llegamos a tener problemas para crearlo con pdfkit, intentaremos con WeasyPrint.
                # WeasyPrint al principio me dio algunos errores, pero queda como alternativa
                HTML(filename=temp_html_path).write_pdf(pdf_filename)
                print(f"PDF exportado: {pdf_filename}")

            except Exception as e_weasy:
                print(f"Error también con WeasyPrint (fallback): {e_weasy}")

        finally:
            # Eliminamos el fichero temporal
            if temp_html_path and os.path.exists(temp_html_path):
                os.remove(temp_html_path)

    # Exportar a HTML
    if export_html:

        html_filename = f"{base_filename}.html"
        with open(html_filename, 'w', encoding='utf-8') as f:
            f.write(html_template)

        print(f"HTML exportado: {html_filename}")


In [17]:
# Función main para ejecutar mi generador de planes alimenticios.
def main():
    # Ruta donde se encuentran mis pdf con menús personales.
    data_path = "./data"
    # Ruta donde guardaremos el índice FAISS
    index_path = "./faiss_index"

    print("-> Cargando documentos PDF desde carpeta")
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        print(f"La carpeta '{data_path}' no existe. Se ha creado. Debes coloca tus menús PDFs aquí.")
        return

    #Cargamos todos los documentos tipo PDF que esten en el directorio.
    docs = cargar_documentos_pdf_directorio(data_path)

    if not docs:
        print("No se encontraron documentos en './data'. Verifica que existen documentos tipo PDF en la carpeta.")
        return

    print(f"-> Se cargaron {len(docs)} documentos.")

    print("-> Chunking (Dividiendo documentos)")
    chunks = dividir_documentos(docs)

    print(f"Número de chunks generados: {len(chunks)}")

    if len(chunks) == 0:
        print("La lista de chunks está vacía. ")
        return

    print("-> Creando u obteniendo índice FAISS existente.")
    vectorstore = None

    try:
        vectorstore = obtener_o_crear_vectorstore(chunks, index_path)

    except ValueError as e:
        print(f"Error al obtener/crear vectorstore: {e}")
        return

    except TypeError as e:
        print(f"Error de tipo al obtener/crear vectorstore: {e}")
        return

    except Exception as e:
        print(f"Error inesperado al construir FAISS: {e}")
        return

    if vectorstore is None:
        print("No se pudo inicializar el vectorstore.")
        return

    print("-> Crear RAG Chain")
    rag_chain = crear_rag(vectorstore)

    if rag_chain is None:
        print("No se pudo inicializar nuestra RAG Chain.")
        return

    print("\n--- ¡Generador de Plan Alimenticio Personalizado! ---")

    # Vamos a mostrar un listado con los diferentes propositos en los que podemos enfocar el  plan para que el usuario seleccione
    propositos = [
        "Resistencia a la insulina",
        "Pérdida de peso",
        "Ganancia muscular",
        "Dieta equilibrada",
        "Control de glucosa",
        "Reducción de inflamación"
    ]

    print("📝 Selecciona el propósito de tu plan alimenticio:")
    for i, proposito in enumerate(propositos):
        print(f"  {i+1}. {proposito}")

    # Propsito del plan alimenticio
    while True:
        try:
            choice_str = input(f"Introduce el número de tu elección (1-{len(propositos)}): ").strip()
            if not choice_str: # Si el usuario no ingresa nada, usa el valor por defecto = 1 "Resistencia a la insulina"
                choice = 1
            else:
                choice = int(choice_str)

            if 1 <= choice <= len(propositos):
                user_prompt = propositos[choice - 1]
                break
            else:
                print(f"⚠️ Por favor, introduce un número entre 1 y {len(propositos)}.")
        except ValueError:
            print("❌ Entrada inválida. Por favor, introduce un número.")

    # Días para el plan
    while True:
        try:
            dias_plan_str = input("📅 ¿De cuántos días quieres tu plan? (1-7 días): ").strip()

            if not dias_plan_str:
                dias_plan = 1 # Si el usuario no ingresa nada, usa el valor por defecto = 1
            else:
                dias_plan = int(dias_plan_str)

            if 1 <= dias_plan <= 7:
                break
            else:
                print("⚠️ Por favor, introduce un número entre 1 y 7.")

        except ValueError:
            print("❌ Entrada inválida. Por favor, introduce un número.")

    # Solicitamos los alimentos que no desea incluir en el plan
    alimentos_descartados = input("🚫 ¿Hay algún alimento que quieras descartar del plan? Indícalos separándos por comas (ejemplo: 'lácteos, gluten, fruta') o simplemente presiona 'enter': ")

    if not alimentos_descartados:
        alimentos_descartados = "Ninguno" # Si el usuario no ingresa nada, usa el valor por defecto = "Ninguno"

    # Cantidad de porciones para personalizar más el plan.
    while True:
        try:
            num_porciones_str = input("👨‍👩‍👧‍👦 ¿Para cuántas porciones deseas las recetas receta? (1, 2, 3): ").strip()
            if not num_porciones_str:
                num_porciones = 1 # Si el usuario no ingresa nada, usa el valor por defecto = 1
            else:
                num_porciones = int(num_porciones_str)

            if num_porciones > 0:
                break
            else:
                print("⚠️ Por favor, introduce un número positivo de porciones.")

        except ValueError:
            print("❌ Entrada inválida. Por favor, introduce un número.")

    print(f"\nGenerando plan personalizdo para '{user_prompt}' de {dias_plan} días, descartando: {alimentos_descartados}, para {num_porciones} porción(es)")
    print("⏳ Esto puede tomar un momento.")

    try:
      # Llamamos al RAG, con las entradas del usuario
        respuesta = rag_chain.invoke({
            "question": user_prompt,
            "dias_plan": dias_plan,
            "alimentos_descartados": alimentos_descartados,
            "num_porciones": num_porciones
        })

        print("\n📄 Respuesta generada:\n")
        # Imprimimos la respuesta en consola
        print(respuesta)

        # Preguntamos al usuario como quiere descargar su plan alimenticio para que pueda conservarlo
        print("\n¿En qué formatos te gustaría descargar tu plan alimenticio?")
        export_pdf = input("¿Exportar a PDF? (s/n): ").strip().lower() != 'n'
        export_html = input("¿Exportar a HTML? (s/n): ").strip().lower() == 's'

        # Exportamos según lo que selecciono el usuario
        exportar_plan(respuesta, export_pdf=export_pdf, export_html=export_html)

    except AuthenticationError:
        print("\nError de autenticación de OpenAI: La API Key proporcionada es inválida.")

    except RateLimitError:
        print("\nError de límite de tarifa de OpenAI: Has excedido tu cuota actual o el límite de solicitudes.")

    except APIError as e:
        print(f"\nError de la API de OpenAI: {e}")

    except Exception as e:
        print(f"\nOcurrió un error inesperado: {e}")


In [19]:
if __name__ == "__main__":
    main()

-> Cargando documentos PDF desde carpeta
-> Se cargaron 35 documentos.
-> Chunking (Dividiendo documentos)
Número de chunks generados: 69
-> Creando u obteniendo índice FAISS existente.
-> Cargando índice FAISS existente desde disco.
-> Crear RAG Chain

--- ¡Generador de Plan Alimenticio Personalizado! ---
📝 Selecciona el propósito de tu plan alimenticio:
  1. Resistencia a la insulina
  2. Pérdida de peso
  3. Ganancia muscular
  4. Dieta equilibrada
  5. Control de glucosa
  6. Reducción de inflamación
Introduce el número de tu elección (1-6): 3
📅 ¿De cuántos días quieres tu plan? (1-7 días): 2
🚫 ¿Hay algún alimento que quieras descartar del plan? Indícalos separándos por comas (ejemplo: 'lácteos, gluten, fruta') o simplemente presiona 'enter': PESCADO
👨‍👩‍👧‍👦 ¿Para cuántas porciones deseas las recetas receta? (1, 2, 3): 1

Generando plan personalizdo para 'Ganancia muscular' de 2 días, descartando: PESCADO, para 1 porción(es)
⏳ Esto puede tomar un momento.

📄 Respuesta generada:

# 